In [1]:
import os
import shutil
from pymongo import MongoClient
import numpy as np
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt
from dotenv import load_dotenv

C:\Users\ASUS\anaconda3\lib\importlib\__init__.py:127: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)


In [2]:
def get_directories(path: str):
    """
    Get all directory names under the specified path.
    
    :param path: The root directory path to search.
    :return: A list of directory names.
    """
    try:
        return [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    except FileNotFoundError:
        print(f"Error: The path '{path}' does not exist.")
        return []
    except PermissionError:
        print(f"Error: Permission denied for path '{path}'.")
        return []
def connect_mongodb(uri,db_name, collection_name):
    # Connect to MongoDB 
    client = MongoClient(uri)
    
    # Select the database and collection
    db = client[db_name]
    collection = db[collection_name]
    
    return collection
def get_ids_with_condition(collection,condition):
    """
    Retrieve an array of `_id` values where "sent-out" is "no data".

    :param collection: MongoDB collection object.
    :return: List of `_id` values matching the condition.
    """
#     condition = {"sent-out": "no data"}
    result = collection.find(condition, {"_id": 1})  # Retrieve only the `_id` field
    return [doc["_id"] for doc in result]
def cluster_text_files(directory, similarity_threshold=0.9):
    """
    Reads text files from a directory, converts text into embeddings using OpenAI, 
    computes cosine similarity, and performs hierarchical clustering.
    Then, for each cluster:
        - Finds the largest file
        - Creates a "similar-to-{largest file name}" directory
        - Moves all remaining files in that cluster to the new directory

    :param directory: Path to the directory containing .txt files.
    :param similarity_threshold: Cosine similarity threshold to define clusters (default is 0.9).
    """
    # Initialize OpenAI embeddings model
    embeddings_model = OpenAIEmbeddings()

    # Collect text content from each .txt file
    file_contents = {}
    file_sizes = {}
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)

        if os.path.isfile(file_path) and filename.endswith(".txt"):
            try:
                with open(file_path, "r", encoding="utf-8") as file:
                    content = file.read().strip()
                    if content:  # Ensure the file is not empty
                        file_contents[filename] = content
                        file_sizes[filename] = os.path.getsize(file_path)  # Get file size
            except Exception as e:
                print(f"Error reading {filename}: {e}")

    # Check if there are no files or only one file
    if len(file_contents) == 0:
        print("No valid text files found. Skipping clustering.")
        return

    if len(file_contents) == 1:
        print(f"Only one file ({list(file_contents.keys())[0]}) found. No clustering needed.")
        return

    # Generate embeddings for each file
    file_names = list(file_contents.keys())
    texts = list(file_contents.values())
    embeddings = np.array([embeddings_model.embed_query(text) for text in texts])

    # Compute cosine similarity matrix
    similarity_matrix = cosine_similarity(embeddings)

    # Convert similarity to distance (1 - similarity)
    distance_matrix = 1 - similarity_matrix

    # Perform hierarchical clustering
    linkage_matrix = linkage(distance_matrix, method='ward')

    # Automatically determine clusters based on similarity threshold
    distance_threshold = 1 - similarity_threshold  # Convert similarity to distance
    clusters = fcluster(linkage_matrix, distance_threshold, criterion='distance')

    # Store cluster assignments
    clustered_files = {}
    for file, cluster_id in zip(file_names, clusters):
        if cluster_id not in clustered_files:
            clustered_files[cluster_id] = []
        clustered_files[cluster_id].append(file)

    print("\n=== Clustered Files (Threshold: {:.0f}%) ===".format(similarity_threshold * 100))
    for cluster, files in clustered_files.items():
        print(f"Cluster {cluster}: {files}")

    # Step 1, Step 2, Step 3: Organize files into "similar-to-{largest file name}" folders
    for cluster, files in clustered_files.items():
        if len(files) > 1:  # Only create a folder if there are multiple files in the cluster
            # Step 1: Find the file with the largest size
            largest_file = max(files, key=lambda f: file_sizes.get(f, 0))
            largest_file_path = os.path.join(directory, largest_file)

            # Step 2: Create the directory "similar-to-{largest file name}"
            cluster_directory = os.path.join(directory, f"similar-to-{largest_file}")
            os.makedirs(cluster_directory, exist_ok=True)

            # Step 3: Move all other files in the cluster to the new directory
            for file in files:
                if file != largest_file:  # Don't move the largest file
                    src_path = os.path.join(directory, file)
                    dest_path = os.path.join(cluster_directory, file)
                    shutil.move(src_path, dest_path)
                    print(f"Moved {file} -> {cluster_directory}/")

In [3]:
# root_traffic_directory_path = r"E:\wearable-capture-traffic\wearable-standalone"
root_traffic_directory_path = r"E:\wearable-capture-traffic\wearable-app"

In [4]:
mongoDB_uri = 'mongodb://192.168.1.14:27017'
mongoDB_database = 'wearable-project' 
# mongoDB_collection = 'wearable-standalone'
mongoDB_collection = 'wearable-app'

In [5]:
array_traffic_directory = get_directories(root_traffic_directory_path)
print(len(array_traffic_directory))

630


In [6]:
# Connect to the MongoDB collection
collection = connect_mongodb(mongoDB_uri,mongoDB_database,mongoDB_collection)
condition = {"sent-out": "no data"}
no_traffic_package_name = get_ids_with_condition(collection,condition)
# print(no_traffic_package_name)
print(len(no_traffic_package_name))

122


In [7]:
array_traffic_package_name = list(set(array_traffic_directory) - set(no_traffic_package_name))
print(array_traffic_package_name)
print(len(array_traffic_package_name))

['com.blinkfitness.app', 'com.komparato.informer.wear', 'rs.nonstopfitness', 'yogaworkouts.dailyyoga.yogafitness', 'de.fitplusint', 'com.FitnessAPPs.superheroWorkouts', 'com.runmifit.android', 'women.workout.female.fitness', 'com.vivefit', 'com.zsmart.zmoofit', 'digifit.virtuagym.client.android', 'com.netpulse.mobile.fitnessforten', 'de.svz', 'com.github.jamesgay.fitnotes', 'com.simplyfit.rowing', 'com.fitifyapps.bwcardio', 'com.strong.legs.workout.muscle.fitness', 'com.skyhealth.fitnessbuddyandroidfree', 'com.hwd.flowfit', 'com.mypthub.colourfit', 'com.polaroidfit.polaroidfit', 'digifit.android.virtuagym.pro.fitnessparksuperclub', 'fit.hplus.hplusfit', 'com.mictale.gpsessentials', 'homeworkoutapp.homeworkout.fitness.workout.loseweight', 'com.womenfitness.workoutbuddy.mea', 'dance.fit.zumba.weightloss.danceburn', 'com.pubu.dafit', 'digifit.android.virtuagym.pro.yababethfit', 'musclemonster.fitness.workout.home.gym.planner', 'ru.starline', 'com.lifetrons.lifetrons.app.fitness', 'com.fit

In [8]:
#. Chat model
# Setup model
# Load environment variables
load_dotenv()

# Retrieve API key
api_key = os.getenv("OPENAI_API_KEY")
# Ensure the API key is correctly set
if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

# Initialize the ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    openai_api_key=api_key  # Ensure you explicitly pass the API key
)

In [9]:
for i in range(len(array_traffic_package_name)):
    print("--------------------- Loop-"+str(i)+" ---------------------")
    package_name = array_traffic_package_name[i]
    print("package_name: ",package_name)
    traffic_directory_path = root_traffic_directory_path + "\\" + package_name
    print("traffic_directory_path: ",traffic_directory_path)
    cluster_text_files(traffic_directory_path, similarity_threshold=0.9)

--------------------- Loop-0 ---------------------
package_name:  com.blinkfitness.app
traffic_directory_path:  E:\wearable-capture-traffic\wearable-app\com.blinkfitness.app


C:\Users\ASUS\AppData\Local\Temp\ipykernel_11596\448174063.py:48: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings_model = OpenAIEmbeddings()



=== Clustered Files (Threshold: 90%) ===
Cluster 3: ['message_0.txt']
Cluster 1: ['message_2.txt']
Cluster 2: ['message_3.txt']
--------------------- Loop-1 ---------------------
package_name:  com.komparato.informer.wear
traffic_directory_path:  E:\wearable-capture-traffic\wearable-app\com.komparato.informer.wear

=== Clustered Files (Threshold: 90%) ===
Cluster 1: ['message_0.txt']
Cluster 2: ['message_2.txt']
--------------------- Loop-2 ---------------------
package_name:  rs.nonstopfitness
traffic_directory_path:  E:\wearable-capture-traffic\wearable-app\rs.nonstopfitness

=== Clustered Files (Threshold: 90%) ===
Cluster 1: ['message_0.txt', 'message_2.txt', 'message_4.txt', 'message_6.txt', 'message_8.txt']
Cluster 3: ['message_1.txt', 'message_7.txt']
Cluster 4: ['message_3.txt']
Cluster 2: ['message_5.txt']
Moved message_0.txt -> E:\wearable-capture-traffic\wearable-app\rs.nonstopfitness\similar-to-message_4.txt/
Moved message_2.txt -> E:\wearable-capture-traffic\wearable-app\


=== Clustered Files (Threshold: 90%) ===
Cluster 1: ['message_0.txt', 'message_1.txt']
Moved message_0.txt -> E:\wearable-capture-traffic\wearable-app\com.fitifyapps.bwcardio\similar-to-message_1.txt/
--------------------- Loop-16 ---------------------
package_name:  com.strong.legs.workout.muscle.fitness
traffic_directory_path:  E:\wearable-capture-traffic\wearable-app\com.strong.legs.workout.muscle.fitness
Only one file (message_0.txt) found. No clustering needed.
--------------------- Loop-17 ---------------------
package_name:  com.skyhealth.fitnessbuddyandroidfree
traffic_directory_path:  E:\wearable-capture-traffic\wearable-app\com.skyhealth.fitnessbuddyandroidfree

=== Clustered Files (Threshold: 90%) ===
Cluster 2: ['message_0.txt', 'message_3.txt']
Cluster 5: ['message_1.txt']
Cluster 3: ['message_2.txt']
Cluster 7: ['message_4.txt']
Cluster 4: ['message_5.txt']
Cluster 1: ['message_6.txt', 'message_8.txt', 'message_9.txt']
Cluster 6: ['message_7.txt']
Moved message_3.txt -> 

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11596\448174063.py:87: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  linkage_matrix = linkage(distance_matrix, method='ward')


...


=== Clustered Files (Threshold: 90%) ===
Cluster 3: ['message_0.txt']
Cluster 1: ['message_1.txt']
Cluster 2: ['message_2.txt']
--------------------- Loop-467 ---------------------
package_name:  fitness.com.senior
traffic_directory_path:  E:\wearable-capture-traffic\wearable-app\fitness.com.senior

=== Clustered Files (Threshold: 90%) ===
Cluster 2: ['message_0.txt']
Cluster 1: ['message_1.txt', 'message_3.txt']
Cluster 3: ['message_5.txt']
Moved message_1.txt -> E:\wearable-capture-traffic\wearable-app\fitness.com.senior\similar-to-message_3.txt/
--------------------- Loop-468 ---------------------
package_name:  com.mauriciotogneri.shoppinglist
traffic_directory_path:  E:\wearable-capture-traffic\wearable-app\com.mauriciotogneri.shoppinglist

=== Clustered Files (Threshold: 90%) ===
Cluster 1: ['message_0.txt', 'message_2.txt']
Cluster 2: ['message_1.txt']
Cluster 3: ['message_10.txt']
Cluster 6: ['message_12.txt']
Cluster 4: ['message_13.txt']
Cluster 9: ['message_3.txt']
Cluster 


=== Clustered Files (Threshold: 90%) ===
Cluster 1: ['message_1.txt']
Cluster 3: ['message_4.txt']
Cluster 2: ['message_5.txt']
--------------------- Loop-477 ---------------------
package_name:  com.ubivelox.uvophone
traffic_directory_path:  E:\wearable-capture-traffic\wearable-app\com.ubivelox.uvophone

=== Clustered Files (Threshold: 90%) ===
Cluster 2: ['message_0.txt', 'message_4.txt']
Cluster 1: ['message_1.txt', 'message_2.txt', 'message_3.txt', 'message_6.txt']
Moved message_0.txt -> E:\wearable-capture-traffic\wearable-app\com.ubivelox.uvophone\similar-to-message_4.txt/
Moved message_2.txt -> E:\wearable-capture-traffic\wearable-app\com.ubivelox.uvophone\similar-to-message_1.txt/
Moved message_3.txt -> E:\wearable-capture-traffic\wearable-app\com.ubivelox.uvophone\similar-to-message_1.txt/
Moved message_6.txt -> E:\wearable-capture-traffic\wearable-app\com.ubivelox.uvophone\similar-to-message_1.txt/
--------------------- Loop-478 ---------------------
package_name:  com.abyx.


=== Clustered Files (Threshold: 90%) ===
Cluster 5: ['message_0.txt', 'message_13.txt']
Cluster 12: ['message_1.txt']
Cluster 3: ['message_10.txt']
Cluster 1: ['message_11.txt']
Cluster 10: ['message_12.txt']
Cluster 9: ['message_14.txt', 'message_5.txt']
Cluster 7: ['message_16.txt']
Cluster 8: ['message_2.txt']
Cluster 6: ['message_3.txt', 'message_8.txt']
Cluster 11: ['message_4.txt']
Cluster 4: ['message_6.txt']
Cluster 2: ['message_7.txt']
Moved message_13.txt -> E:\wearable-capture-traffic\wearable-app\com.o2fitt\similar-to-message_0.txt/
Moved message_5.txt -> E:\wearable-capture-traffic\wearable-app\com.o2fitt\similar-to-message_14.txt/
Moved message_8.txt -> E:\wearable-capture-traffic\wearable-app\com.o2fitt\similar-to-message_3.txt/
--------------------- Loop-490 ---------------------
package_name:  digifit.android.virtuagym.pro.buzzgymsuperclub
traffic_directory_path:  E:\wearable-capture-traffic\wearable-app\digifit.android.virtuagym.pro.buzzgymsuperclub

=== Clustered Fi


=== Clustered Files (Threshold: 90%) ===
Cluster 3: ['message_0.txt']
Cluster 4: ['message_1.txt']
Cluster 5: ['message_2.txt']
Cluster 6: ['message_4.txt']
Cluster 1: ['message_5.txt', 'message_7.txt']
Cluster 2: ['message_6.txt']
Moved message_7.txt -> E:\wearable-capture-traffic\wearable-app\com.huawo.hawofit\similar-to-message_5.txt/
--------------------- Loop-506 ---------------------
package_name:  com.pushpress.punchkingfitnesssunset
traffic_directory_path:  E:\wearable-capture-traffic\wearable-app\com.pushpress.punchkingfitnesssunset

=== Clustered Files (Threshold: 90%) ===
Cluster 1: ['message_0.txt', 'message_8.txt']
Cluster 2: ['message_10.txt', 'message_5.txt']
Cluster 3: ['message_2.txt']
Cluster 5: ['message_7.txt']
Cluster 4: ['message_9.txt']
Moved message_8.txt -> E:\wearable-capture-traffic\wearable-app\com.pushpress.punchkingfitnesssunset\similar-to-message_0.txt/
Moved message_5.txt -> E:\wearable-capture-traffic\wearable-app\com.pushpress.punchkingfitnesssunset\s


=== Clustered Files (Threshold: 90%) ===
Cluster 1: ['message_1.txt']
Cluster 2: ['message_2.txt']
--------------------- Loop-517 ---------------------
package_name:  com.highfitnesslive
traffic_directory_path:  E:\wearable-capture-traffic\wearable-app\com.highfitnesslive
Only one file (message_1.txt) found. No clustering needed.
--------------------- Loop-518 ---------------------
package_name:  app.EnForma.fitcoapp.net
traffic_directory_path:  E:\wearable-capture-traffic\wearable-app\app.EnForma.fitcoapp.net

=== Clustered Files (Threshold: 90%) ===
Cluster 2: ['message_0.txt', 'message_2.txt', 'message_3.txt', 'message_4.txt', 'message_6.txt']
Cluster 1: ['message_1.txt', 'message_5.txt']
Moved message_0.txt -> E:\wearable-capture-traffic\wearable-app\app.EnForma.fitcoapp.net\similar-to-message_2.txt/
Moved message_3.txt -> E:\wearable-capture-traffic\wearable-app\app.EnForma.fitcoapp.net\similar-to-message_2.txt/
Moved message_4.txt -> E:\wearable-capture-traffic\wearable-app\app.